In [22]:
import pandas as pd
import numpy as np
import re

In [23]:
def extract_date(s):
    return re.search("([0-9]{2}.[0-9]{2}.[0-9]{4})", s).group(0)

In [55]:
def remove_starting_numbers(s):
    return re.search("([0-9]{1,10}: (.*))", s).group(2)

In [4]:
erp_raw = pd.read_excel("ведомостьЕРПоктябрь.xlsx",skiprows=8, header=0, skipfooter=1)

In [5]:
erp = erp_raw.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 4', 'Unnamed: 5'], axis=1)

In [6]:
erp.rename({'Unnamed: 0':'wh', 'Unnamed: 3':'product', 'Unnamed: 6':'doc', 'Приход':'in', 'Расход':'out'}, axis=1, inplace=True)

In [7]:
erp['doc_date'] = erp['doc'].apply(extract_date)

In [8]:
erp.fillna(0.0, inplace=True)

**в ВА кофе:**  
Группировка: Склад-Товар  
*фильтры:*  
{Категория.Склад} в списке ('Плаза- Телеком - СКЛАД (Ингредиенты)', 'СКЛАД (Ингредиенты)  Санкт-Петербург');  
{Количество.Приход} <> 0 OR {Количество.Расход} <> 0

In [25]:
va_coff_raw = pd.read_excel("оборотыВАкофеОктябрь.xls",skiprows=1, header=0, skipfooter=1)

In [26]:
va_coff_raw.head()

,Склад/Товар,Начальный остаток,Приход,Расход,Остаток,Начальный остаток.1,Приход.1,Расход.1,Остаток.1
0,Плаза- Телеком - СКЛАД (Ингредиенты),587528.0,633394.9,457828.9,763094.0,4.355314e+06,1.144578e+07,9.654421e+06,6.146673e+06
1,Вода родниковая Аквагор 19л (Маркируемая),7011.0,59888.0,60401.0,6498.0,1.746558e+05,8.722845e+05,8.386363e+05,2.083040e+05
2,001: Выдача оператору №3 от 02.10.2023 1...,7011.0,0.0,266.0,6745.0,1.746558e+05,0.000000e+00,3.878000e+03,1.707778e+05
3,002: Выдача оператору №4 от 02.10.2023 1...,6745.0,0.0,285.0,6460.0,1.707778e+05,0.000000e+00,4.155000e+03,1.666228e+05
4,003: Выдача оператору №6 от 02.10.2023 1...,6460.0,0.0,152.0,6308.0,1.666228e+05,0.000000e+00,2.216000e+03,1.644068e+05


In [27]:
va_coff = va_coff_raw.drop(['Начальный остаток', 'Остаток',
       'Начальный остаток.1', 'Приход.1', 'Расход.1', 'Остаток.1'], axis='columns')
va_coff.rename({'Склад/Товар':'doc', 'Приход':'in', 'Расход':'out'}, axis=1, inplace=True)

In [28]:
# новая колонка с кол-вом начальных (группировочных) пробелов
va_coff['ident_number'] = va_coff['doc'].str.len() - va_coff['doc'].str.lstrip().str.len()

In [29]:
# по кол-ву пробелов определяем, к какой группировке относится значение
va_coff['wh'] = np.where(va_coff['ident_number'] == 0, va_coff['doc'], np.NaN)
va_coff['product'] = np.where(va_coff['ident_number'] == 3, va_coff['doc'], np.NaN)

In [30]:
# заполняем пропуски вышестоящим значением
va_coff['wh'].ffill(inplace=True)
va_coff['product'].ffill(inplace=True)

In [31]:
# убираем те строки, по которым была группировка, оставляя только с регистратором
# т.к. данные о группировке уже есть в колонках
va_coff = va_coff.drop(va_coff[va_coff.ident_number != 6].index).reset_index(drop=True)
va_coff = va_coff.drop('ident_number', axis='columns')

In [33]:
va_coff['doc_date'] = va_coff['doc'].apply(extract_date)

In [34]:
va_coff['doc'] = va_coff['doc'].str.lstrip()
va_coff['product'] = va_coff['product'].str.lstrip()

In [56]:
# убрать цифры перед названием регистратора (05: Выдача оператору...)
va_coff['doc'] = va_coff['doc'].apply(remove_starting_numbers)

In [20]:
erp.wh.unique()

array(['Товары для снэков (МСК Автомоторная 5Б)',
       'Товары для снэков (СПБ Фаянсовая 22)',
       'Ингридиенты Кофе (СПБ Фаянсовая 22)',
       'Ингридиенты Кофе (МСК Автомоторная 5Б)'], dtype=object)

In [21]:
va_coff.wh.unique()

array(['Плаза- Телеком - СКЛАД (Ингредиенты)',
       'СКЛАД (Ингредиенты)  Санкт-Петербург'], dtype=object)

In [22]:
# key-value pair to remap.
wh_dict = {'Плаза- Телеком - СКЛАД (Ингредиенты)' : 'Ингридиенты Кофе (МСК Автомоторная 5Б)',
           'СКЛАД (Ингредиенты)  Санкт-Петербург' : 'Ингридиенты Кофе (СПБ Фаянсовая 22)'}
va_coff = va_coff.replace({'wh': wh_dict})

In [23]:
va_coff.wh.unique()

array(['Ингридиенты Кофе (МСК Автомоторная 5Б)',
       'Ингридиенты Кофе (СПБ Фаянсовая 22)'], dtype=object)

In [24]:
# выбрать только склады кофе
erp_coff = erp[(erp.wh == 'Ингридиенты Кофе (МСК Автомоторная 5Б)') | 
              (erp.wh == 'Ингридиенты Кофе (СПБ Фаянсовая 22)')]

In [25]:
erp_coff.head()

,wh,product,doc,in,out,doc_date
3340,Ингридиенты Кофе (СПБ Фаянсовая 22),Стакан картонный 330 мл «ВкусВилл»,Перемещение товаров PTEP-000130 от 02.10.2023 ...,0.0,1000.0,02.10.2023
3341,Ингридиенты Кофе (СПБ Фаянсовая 22),Кофе ВВ,Перемещение товаров PTEP-000130 от 02.10.2023 ...,0.0,4.0,02.10.2023
3342,Ингридиенты Кофе (СПБ Фаянсовая 22),Вода родниковая Аквагор 19л (Маркируемая),Перемещение товаров PTEP-000130 от 02.10.2023 ...,0.0,209.0,02.10.2023
3473,Ингридиенты Кофе (СПБ Фаянсовая 22),Вода родниковая Аквагор 19л (Маркируемая),Приходный ордер на товары EP-00001334 от 02.10...,50.0,0.0,02.10.2023
3530,Ингридиенты Кофе (МСК Автомоторная 5Б),"Ловушка от насекомых, шт",Ордер на отражение недостач товаров EP-0000001...,0.0,18.0,02.10.2023


In [35]:
df_merged = pd.merge(va_coff, erp_coff, how='outer', on=['wh', 'product', 'in', 'out', 'doc_date'],
                     suffixes=('_va', '_erp'), indicator=True)

In [40]:
df_merged.sort_values(['wh', 'doc_date', 'doc_va', 'product']).to_excel('sort_coff.xlsx', index=False)

In [36]:
df_merged.to_excel('merge_coff.xlsx', index=False)

In [54]:
re.search("([0-9]{1,10}: (.*))", '021: Выдача оператору №28 от 04.10.2023 17:04:05').group(2)

'Выдача оператору №28 от 04.10.2023 17:04:05'